Problem Statement: Make sure to state the metric we're using to evaluate fit!  In our case, it's RMSE.

# Imports

In [1]:
# Import packages
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.linear_model import LinearRegression
from sklearn import metrics

# Data Cleaning Functions

Many of the variables in this dataset contain large numbers of missing values.  For example, according to the [data dictionary](http://jse.amstat.org/v19n3/decock/DataDocumentation.txt) `Alley` is supposed to take on 3 values: 'Grvl', 'Pave', and 'NA.'  However, the vast majority of observations are have missing values in this column.  But: we see that among all observations in the data set, *only the values 'Grvl' and 'Pave' appear in the column `Alley`*!  This implies that what the data dictionary indicates as "NA" means "We left this value blank."

So to eliminate missing data problems, we'll want to automatically fill in with "DNE" (Does Not Exist) any missing values that have this property.  That is, if a categorical variable is listed in the data dictionary as being able to take on the value "NA" and furthermore has missing values, then we'll check whether that variable has achieved the full number of categories it's supposed to (according to the data dictionary).  If not, then we'll automatically fill its missing values with "DNE".

In [63]:
#Create a dictionary listing how many distinct values each of the categorical variables
#is supposed to be able to take on.  If fewer than this many are achieved, then that's
#a sign that this variable's "missing values" are just instances of "NA".
categorical_nums = {'Alley': 3, 'Mas Vnr Type': 5, 'Bsmt Qual': 6, 'Bsmt Cond': 6, 'Bsmt Exposure': 5,
                'BsmtFin Type 1': 7, 'BsmtFin Type 2': 7, 'Fireplace Qu': 6, 'Garage Type': 7,
                'Garage Finish': 4, 'Garage Qual': 6, 'Garage Cond': 6, 'Pool QC': 5, 'Fence': 5,
                'Misc Feature': 6,
                'Garage Yr Blt': np.Inf,
                }
#Some homes may not have a garage, so its "year built" may be missing for this reason, even though
#this variable is numeric (not categorical).  But we want to fill these with "NA" rather than 0

categorical_names = categorical_nums.keys()

In [64]:
#Also, we'll want to create a list of those numeric variable names that are likely to be ones
#where a missing value would be equivalent to a "zero."  For example, "Lot Frontage" being
#missing probably means that there are just 0 feet of street connected to the property.
numeric_names = ['Lot Frontage', 'Lot Area', 'Mas Vnr Area', 'BsmtFin SF 1', 'BsmtFin SF 2',
                 'Bsmt Unf SF', 'Total Bsmt SF', '1st Flr SF', '2nd Flr SF', 'Low Qual Fin SF',
                 'Gr Liv Area', 'Bsmt Full Bath', 'Bsmt Half Bath', 'Full Bath', 'Half Bath',
                 'Bedroom AbvGr', 'Kitchen AbvGr', 'TotRms AbvGrd', 'Fireplaces', 'Garage Cars',
                 'Garage Area', 'Wood Deck SF', 'Open Porch SF', 'Enclosed Porch', '3Ssn Porch',
                 'Screen Porch', 'Pool Area', 'Misc Val', 'SalePrice']

# Data Import

In [4]:
#Import data
df = pd.read_csv('./datasets/train.csv')

# Checking Missing Values

In [5]:
#Look for missing values
variables_with_missing = df.isnull().sum().index
missing_nos = df.isnull().sum().values

for i in range(len(variables_with_missing)):
    if missing_nos[i] != 0:
        print(f'{missing_nos[i]} missing values from variable {variables_with_missing[i]}')

330 missing values from variable Lot Frontage
1911 missing values from variable Alley
22 missing values from variable Mas Vnr Type
22 missing values from variable Mas Vnr Area
55 missing values from variable Bsmt Qual
55 missing values from variable Bsmt Cond
58 missing values from variable Bsmt Exposure
55 missing values from variable BsmtFin Type 1
1 missing values from variable BsmtFin SF 1
56 missing values from variable BsmtFin Type 2
1 missing values from variable BsmtFin SF 2
1 missing values from variable Bsmt Unf SF
1 missing values from variable Total Bsmt SF
2 missing values from variable Bsmt Full Bath
2 missing values from variable Bsmt Half Bath
1000 missing values from variable Fireplace Qu
113 missing values from variable Garage Type
114 missing values from variable Garage Yr Blt
114 missing values from variable Garage Finish
1 missing values from variable Garage Cars
1 missing values from variable Garage Area
114 missing values from variable Garage Qual
114 missing val

## General Approach to Missing Data
Looking at the [data description](http://jse.amstat.org/v19n3/decock/DataDocumentation.txt), we see that some variables may be recorded as "NA" or "None" to mean *not* that the data set is missing an observation, but instead that a certain feature is missing.  For example, the variable "Alley" is supposed to take on 3 values: 'Grvl', 'Pave' and 'NA'.  But we see that all the non-missing values are 'Grvl' or 'Pave' (see cell below).  So this means that most likely these values were not *missing* but were recorded as empty if they met certain criteria.

Our general approach to checking for missing values will be: for each variable, first check the data description for whether "NA" or "None" (etc.) could represent a category rather than a missing value.  If so, replace all instances of that missing value with the category name it's supposed to be.  If not, we'll deal with the missing entry as though it is actually missing.

Secondly, looking at the list of missing values in the previous cell, we see that often times several variables relate to the same part of the house.  For example, there are many variables with the word `Bsmt` (basement) in them.  Many of these variables seem to be *missing exactly 55 entries, or just a few more than 55 entries*.  Looking at the data description, wherein we find that "NA" means "No Basement", it is plausible to think that all 55 of the missing values for each of the `Bsmt` variables come from the same 55 houses.  We'll verify that this is the case before replacing these variables' 55 missing values with the string `no_bsmt`.

But what do we do with the `Bsmt` variables that are missing *more than 55 entries*?  For example, `Bsmt Exposure` is missing 58 values, not 55.  If we applied the same approach to this variable as we did to the other `Bsmt` variables that were missing exactly 55 entries, we would essentially be concluding that all 58 of these missing entries correspond to homes with no basements (and not to actual missing data).  But it is not plausible that *more than 55 of the houses in the data set had no basement*, since several of the `Bsmt` variables were missing *only 55 entries, and the same 55 in each of these variables*.  It therefore seems more likely that 55 of the 58 missing values in `Bsmt Exposure` are those same 55 homes that were elsewhere recorded to have no basement, while the other 3 missing values are actually missing.  So assuming that exactly 55 of the missing values for `Bsmt Exposure` match up to the same 55 houses from before, we'll replace only *these* missing values with the string `no_bsmt`, while leaving the other 3 values missing for now.

Similar remarks apply to the many variables relating to the garage that are missing around 114 entries.

## Basements
We can see that 3 of the `Bsmt` variables are missing 55 values.  Let's check that these are all the same 55 rows in each case:

In [6]:
condition = df['Bsmt Qual'].isnull() | df['Bsmt Cond'].isnull() | df['BsmtFin Type 1'].isnull()
#Look at those rows with ANY of these three variables missing

len(df[condition])
#This should equal 55 if all the homes missing one of these variables are missing the other two as well

55

So these 3 variables' 55 missing entries all correspond to the same 55 homes.  Let's check if these same 55 are missing from `Bsmt Exposure` and `BsmtFin Type 2`:

In [7]:
#Bsmt Exposure
condition1 = df['Bsmt Qual'].isnull() & df['Bsmt Exposure'].isnull() & df['BsmtFin Type 2'].isnull()
#Look at those rows with ALL THREE of these variables missing

len(df[condition1])
#This should equal 55 if all 55 of those homes that are missing Bsmt Qual also have  missing values
#in these other two variables

55

So the variables `Bsmt Exposure` and `BsmtFin Type 2` are missing the same 55 homes as those missing from `Bsmt Qual`, plus 3 more for `Bsmt Exposrue` and 1 more for `BsmtFin Type 2`.  Let's fill in the 55 for all of these 5 variables:

In [8]:
#Find the indices of the 55 houses with missing basements
missing_bsmts = df[df['Bsmt Qual'].isnull()].index

#Make sure there are 55 of them
len(missing_bsmts)

55

In [9]:
missing_bsmts

Int64Index([  12,   93,  114,  146,  183,  240,  249,  256,  390,  437,  485,
             499,  514,  580,  581,  616,  635,  642,  696,  700,  737,  772,
             790,  794,  924,  933,  954, 1007, 1022, 1049, 1089, 1098, 1125,
            1157, 1211, 1222, 1252, 1256, 1270, 1327, 1331, 1377, 1387, 1428,
            1481, 1594, 1620, 1681, 1684, 1686, 1859, 1875, 1889, 1933, 2010],
           dtype='int64')

In [10]:
#Fill the missing values corresponding to these 55 homes, in all bsmt columns missing at least 55 values
bsmts = ['Bsmt Qual', 'Bsmt Cond', 'BsmtFin Type 1', 'Bsmt Exposure', 'BsmtFin Type 2']

for col in bsmts:
    df.iloc[missing_bsmts, df.columns.get_loc(col)] = 'no_bsmt'

In [11]:
for col in bsmts:
    print(f"Number of rows with missing {col} value: {len(df[df[col].isnull()])}")

Number of rows with missing Bsmt Qual value: 0
Number of rows with missing Bsmt Cond value: 0
Number of rows with missing BsmtFin Type 1 value: 0
Number of rows with missing Bsmt Exposure value: 3
Number of rows with missing BsmtFin Type 2 value: 1


In [12]:
#Check that a row had its missing value replaced correctly
df['Bsmt Qual'][12]

'no_bsmt'

In [13]:
df[df['Bsmt Exposure'].isnull()].index

Int64Index([1456, 1547, 1997], dtype='int64')

In [14]:
#Which observations now have missing values?  Let's examine them and record their indices
drops = df[df['Bsmt Exposure'].isnull()].index
drop_indices = list(drops)
df.iloc[ drops ]

,Id,PID,MS SubClass,MS Zoning,Lot Frontage,Lot Area,Street,Alley,Lot Shape,Land Contour,...,Screen Porch,Pool Area,Pool QC,Fence,Misc Feature,Misc Val,Mo Sold,Yr Sold,Sale Type,SalePrice
1456,1797,528458090,60,FV,81.0,10411,Pave,NaN,Reg,Lvl,...,0,0,NaN,NaN,NaN,0,7,2007,New,212109
1547,67,528445060,20,RL,73.0,8987,Pave,NaN,Reg,Lvl,...,0,0,NaN,NaN,NaN,0,5,2010,WD,221500
1997,2780,907194130,60,RL,65.0,14006,Pave,NaN,IR1,Lvl,...,0,0,NaN,NaN,NaN,0,2,2006,WD,192500


In [15]:
drop = df[df['BsmtFin Type 2'].isnull()].index
drop_indices.extend(list(drop))
df.iloc[ drop ]

,Id,PID,MS SubClass,MS Zoning,Lot Frontage,Lot Area,Street,Alley,Lot Shape,Land Contour,...,Screen Porch,Pool Area,Pool QC,Fence,Misc Feature,Misc Val,Mo Sold,Yr Sold,Sale Type,SalePrice
1147,445,528142130,20,RL,85.0,10655,Pave,NaN,IR1,Lvl,...,0,0,NaN,NaN,NaN,0,10,2009,WD,284000


In [16]:
#Check that we've recorded the right indices to drop later
drop_indices

[1456, 1547, 1997, 1147]

In summary: none of the 3 observations with missing `Bsmt Exposure` values are the same as the one missing its `BsmtFin Type 2` value.

In any case, we'll want to drop these values later, so we'll keep track of their indices with `drop_indices`.

Now let's move on to the other `Bsmt` variables that were missing some observations:

In [17]:
#Now examine other 'Bmst' varialbes that had missing values (each had 1)
bsmts2 = ['BsmtFin SF 1', 'BsmtFin SF 2', 'Bsmt Unf SF', 'Total Bsmt SF']
for col in bsmts2:
    print(f"Number of rows with missing {col} value: {len(df[df[col].isnull()])}")

Number of rows with missing BsmtFin SF 1 value: 1
Number of rows with missing BsmtFin SF 2 value: 1
Number of rows with missing Bsmt Unf SF value: 1
Number of rows with missing Total Bsmt SF value: 1


In [18]:
#Are these missing values all the same one?
df[ df['BsmtFin SF 1'].isnull() & df['BsmtFin SF 2'].isnull() & df['Bsmt Unf SF'].isnull() & df['Total Bsmt SF'].isnull() ]

,Id,PID,MS SubClass,MS Zoning,Lot Frontage,Lot Area,Street,Alley,Lot Shape,Land Contour,...,Screen Porch,Pool Area,Pool QC,Fence,Misc Feature,Misc Val,Mo Sold,Yr Sold,Sale Type,SalePrice
1327,1342,903230120,20,RM,99.0,5940,Pave,NaN,IR1,Lvl,...,0,0,NaN,MnPrv,NaN,0,4,2008,ConLD,79000


In [19]:
#They are! How about these other cloumn names that were missing exactly 2 values?
bsmts3 = ['Bsmt Full Bath', 'Bsmt Half Bath']
for col in bsmts3:
    print(f"Number of rows with missing {col} value: {len(df[df[col].isnull()])}")
    
df[ df['Bsmt Full Bath'].isnull() & df['Bsmt Half Bath'].isnull() ]

Number of rows with missing Bsmt Full Bath value: 2
Number of rows with missing Bsmt Half Bath value: 2


,Id,PID,MS SubClass,MS Zoning,Lot Frontage,Lot Area,Street,Alley,Lot Shape,Land Contour,...,Screen Porch,Pool Area,Pool QC,Fence,Misc Feature,Misc Val,Mo Sold,Yr Sold,Sale Type,SalePrice
616,1498,908154080,20,RL,123.0,47007,Pave,NaN,IR1,Lvl,...,0,0,NaN,NaN,NaN,0,7,2008,WD,284700
1327,1342,903230120,20,RM,99.0,5940,Pave,NaN,IR1,Lvl,...,0,0,NaN,MnPrv,NaN,0,4,2008,ConLD,79000


In summary: Observation number 1327 is missing all 6 of the values listed in `bsmts2` and `bsmts3`, while observation number 616 is only missing both of those values in `bsmts3`.

Meanwhile, 4 other observations were missing their `Bsmt Exposure` or `BsmtFin Type 2` values; we already recorded their indices in our `drop_indices` list.

I think we can safely drop these 6 observations, as our data set has over 2000 observations.

In [20]:
#Add these last two to our list of indices to drop
drop_indices.extend( list( df[ df['Bsmt Full Bath'].isnull() ].index) )
drop_indices

[1456, 1547, 1997, 1147, 616, 1327]

In [21]:
len(df)

2051

In [22]:
#Drop these 6 observations
df.drop(index=drop_indices, inplace=True)
len(df)

2045

In [23]:
#Check the null counts to make sure nothing is still missing
for col in bsmts + bsmts2 + bsmts3:
    print(f"Number of rows with missing {col} value: {len(df[df[col].isnull()])}")

Number of rows with missing Bsmt Qual value: 0
Number of rows with missing Bsmt Cond value: 0
Number of rows with missing BsmtFin Type 1 value: 0
Number of rows with missing Bsmt Exposure value: 0
Number of rows with missing BsmtFin Type 2 value: 0
Number of rows with missing BsmtFin SF 1 value: 0
Number of rows with missing BsmtFin SF 2 value: 0
Number of rows with missing Bsmt Unf SF value: 0
Number of rows with missing Total Bsmt SF value: 0
Number of rows with missing Bsmt Full Bath value: 0
Number of rows with missing Bsmt Half Bath value: 0


In [24]:
#Re-index after dropping (failing to do so causes errors below)
df.reset_index(inplace=True,drop=True)
df.head()

,Id,PID,MS SubClass,MS Zoning,Lot Frontage,Lot Area,Street,Alley,Lot Shape,Land Contour,...,Screen Porch,Pool Area,Pool QC,Fence,Misc Feature,Misc Val,Mo Sold,Yr Sold,Sale Type,SalePrice
0,109,533352170,60,RL,NaN,13517,Pave,NaN,IR1,Lvl,...,0,0,NaN,NaN,NaN,0,3,2010,WD,130500
1,544,531379050,60,RL,43.0,11492,Pave,NaN,IR1,Lvl,...,0,0,NaN,NaN,NaN,0,4,2009,WD,220000
2,153,535304180,20,RL,68.0,7922,Pave,NaN,Reg,Lvl,...,0,0,NaN,NaN,NaN,0,1,2010,WD,109000
3,318,916386060,60,RL,73.0,9802,Pave,NaN,Reg,Lvl,...,0,0,NaN,NaN,NaN,0,4,2010,WD,174000
4,255,906425045,50,RL,82.0,14235,Pave,NaN,IR1,Lvl,...,0,0,NaN,NaN,NaN,0,3,2010,WD,138500


## Garages

In [25]:
garages = ['Garage Qual', 'Garage Cond', 'Garage Finish', 'Garage Yr Blt', 'Garage Type']

In [26]:
df[garages + ['Garage Cars', 'Garage Area']].isnull().sum()

Garage Qual      114
Garage Cond      114
Garage Finish    114
Garage Yr Blt    114
Garage Type      113
Garage Cars        1
Garage Area        1
dtype: int64

In [27]:
#Check if all 114 are the same homes
len(df[ df['Garage Qual'].isnull() & df['Garage Cond'].isnull() & df['Garage Finish'].isnull() & df['Garage Yr Blt'].isnull() ])

114

In [28]:
#Check if those 113 are a subset of the 114
len(df[ df['Garage Qual'].isnull() & df['Garage Type'].isnull() ])

113

In [29]:
#Which data point is the one that's missing all the Garage values EXCEPT `Garage Type`?
df[ df['Garage Qual'].isnull() & ~df['Garage Type'].isnull() ]

,Id,PID,MS SubClass,MS Zoning,Lot Frontage,Lot Area,Street,Alley,Lot Shape,Land Contour,...,Screen Porch,Pool Area,Pool QC,Fence,Misc Feature,Misc Val,Mo Sold,Yr Sold,Sale Type,SalePrice
1707,2237,910201180,70,RM,50.0,9060,Pave,NaN,Reg,Lvl,...,0,0,NaN,MnPrv,NaN,0,3,2007,WD,150909


In [30]:
#Is this the same one observation that's missing both `Garage Cars` and `Garage Area`?
df[df['Garage Cars'].isnull() & df['Garage Area'].isnull()]

,Id,PID,MS SubClass,MS Zoning,Lot Frontage,Lot Area,Street,Alley,Lot Shape,Land Contour,...,Screen Porch,Pool Area,Pool QC,Fence,Misc Feature,Misc Val,Mo Sold,Yr Sold,Sale Type,SalePrice
1707,2237,910201180,70,RM,50.0,9060,Pave,NaN,Reg,Lvl,...,0,0,NaN,MnPrv,NaN,0,3,2007,WD,150909


In [31]:
#It is! So we'll just drop that one observation, then fill the other 113 with "no_garage"

#First, get the index of the one observation to drop:
drop_index = df[df['Garage Cars'].isnull() & df['Garage Area'].isnull()].index
drop_index

Int64Index([1707], dtype='int64')

In [32]:
#Drop this observation
df.drop(index=drop_index, inplace=True)

#Check that it was dropped
df[df['Garage Cars'].isnull() & df['Garage Area'].isnull()]

,Id,PID,MS SubClass,MS Zoning,Lot Frontage,Lot Area,Street,Alley,Lot Shape,Land Contour,...,Screen Porch,Pool Area,Pool QC,Fence,Misc Feature,Misc Val,Mo Sold,Yr Sold,Sale Type,SalePrice


In [33]:
#Reset the index first, so we don't run into issues
#When I didn't do this, an indexing issue caused the code below to not work correctly
df.reset_index(inplace=True,drop=True)
df.head()

,Id,PID,MS SubClass,MS Zoning,Lot Frontage,Lot Area,Street,Alley,Lot Shape,Land Contour,...,Screen Porch,Pool Area,Pool QC,Fence,Misc Feature,Misc Val,Mo Sold,Yr Sold,Sale Type,SalePrice
0,109,533352170,60,RL,NaN,13517,Pave,NaN,IR1,Lvl,...,0,0,NaN,NaN,NaN,0,3,2010,WD,130500
1,544,531379050,60,RL,43.0,11492,Pave,NaN,IR1,Lvl,...,0,0,NaN,NaN,NaN,0,4,2009,WD,220000
2,153,535304180,20,RL,68.0,7922,Pave,NaN,Reg,Lvl,...,0,0,NaN,NaN,NaN,0,1,2010,WD,109000
3,318,916386060,60,RL,73.0,9802,Pave,NaN,Reg,Lvl,...,0,0,NaN,NaN,NaN,0,4,2010,WD,174000
4,255,906425045,50,RL,82.0,14235,Pave,NaN,IR1,Lvl,...,0,0,NaN,NaN,NaN,0,3,2010,WD,138500


In [34]:
#Find the indices of the 113 houses with missing basements
missing_garages = df[df['Garage Qual'].isnull()].index

#Make sure there are 113 of them
len(missing_garages)

113

In [35]:
#Fill the remaining 113 garage observations with 'no_garage'

for col in garages:
    df.iloc[missing_garages, df.columns.get_loc(col)] = 'no_garage'

In [36]:
for col in garages:
    print(f"Number of rows with missing {col} value: {len(df[df[col].isnull()])}")

Number of rows with missing Garage Qual value: 0
Number of rows with missing Garage Cond value: 0
Number of rows with missing Garage Finish value: 0
Number of rows with missing Garage Yr Blt value: 0
Number of rows with missing Garage Type value: 0


## Masonry Veneer

In [37]:
masonry = ['Mas Vnr Type', 'Mas Vnr Area']
for col in masonry:
    print(f"Number of rows with missing {col} value: {len(df[df[col].isnull()])}")

Number of rows with missing Mas Vnr Type value: 22
Number of rows with missing Mas Vnr Area value: 22


In [38]:
#Are these all the same 22 data points?
len(df[ df['Mas Vnr Type'].isnull() & df['Mas Vnr Area'].isnull() ])

22

In [39]:
#They are! That means they're all just homes that lack any masonry venerr.
#So let's fill in the missing `Vnr Type` variables with 'no_vnr'
#and then fill in the missing 'Vnr Area' with 0

#First, get the indices
no_vnr = df[df['Mas Vnr Type'].isnull()].index
len(no_vnr)

22

In [40]:
#Fill in the missing values
df.iloc[no_vnr, df.columns.get_loc('Mas Vnr Type')] = 'no_vnr'
df.iloc[no_vnr, df.columns.get_loc('Mas Vnr Area')] = 0

for col in masonry:
    print(f"Number of rows with missing {col} value: {len(df[df[col].isnull()])}")

Number of rows with missing Mas Vnr Type value: 0
Number of rows with missing Mas Vnr Area value: 0


## Alley Access

In [41]:
for col in ['Alley']:
    print(f"Number of rows with missing {col} value: {len(df[df[col].isnull()])}")

Number of rows with missing Alley value: 1904


In [42]:
# From data dictionary, 'Alley' can take 3 values, one of which is "NA".
# Does NA mean missing, or is it a string?
df['Alley'].value_counts()

Grvl    85
Pave    55
Name: Alley, dtype: int64

In [43]:
#So "NA" means missing.  Thus, missing values in this variable mean
#"no alley access."  Let's replace the missing values with 'no_alley_access':
df["Alley"].fillna("no_alley_access", inplace=True);

In [44]:
df['Alley'].value_counts()

no_alley_access    1904
Grvl                 85
Pave                 55
Name: Alley, dtype: int64

In [45]:
for col in ['Alley']:
    print(f"Number of rows with missing {col} value: {len(df[df[col].isnull()])}")

Number of rows with missing Alley value: 0
